In [1]:
import os
import argparse
import re

import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def sorted_alphanum(file_list_ordered):
    """
    Sorts the list alphanumerically
    Args:
        file_list_ordered (list): list of files to be sorted
    Return:
        sorted_list (list): input list sorted alphanumerically
    """
    def convert(text):
        return int(text) if text.isdigit() else text

    def alphanum_key(key):
        return [convert(c) for c in re.split('([0-9]+)', key)]

    sorted_list = sorted(file_list_ordered, key=alphanum_key)

    return sorted_list
    
def get_file_list(path, extension=None):
    """
    Build a list of all the files in the provided path
    Args:
        path (str): path to the directory 
        extension (str): only return files with this extension
    Return:
        file_list (list): list of all the files (with the provided extension) sorted alphanumerically
    """
    if extension is None:
        file_list = [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    else:
        file_list = [
            os.path.join(path, f)
            for f in os.listdir(path)
            if os.path.isfile(os.path.join(path, f)) and os.path.splitext(f)[1] == extension
        ]
    file_list = sorted_alphanum(file_list)

    return file_list

In [3]:
def scene_flow_EPE_np(pred, labels):
    error = np.sqrt(np.sum((pred - labels)**2, 1) + 1e-20)

    gtflow_len = np.sqrt(np.sum(labels*labels, 1) + 1e-20) # B,N
    acc1 = np.sum(np.logical_or((error <= 0.05), (error/gtflow_len <= 0.05)))
    acc2 = np.sum(np.logical_or((error <= 0.1), (error/gtflow_len <= 0.1)))
    acc3 = np.sum(np.logical_or((error > 0.3), (error/gtflow_len > 0.1)))

    num = pred.shape[0]
    acc1 = acc1 / num
    acc2 = acc2 / num
    acc3 = acc3 / num

    EPE = np.mean(error)
    return EPE, acc1, acc2, acc3

In [4]:
def scene_flow_MAE_np(pred, labels):
    pred_norm = np.sqrt(np.sum(pred * pred, axis=1))
    labels_norm = np.sqrt(np.sum(labels * labels, axis=1))
    cos_theta = (np.sum(pred * labels, axis=1))/ (pred_norm * labels_norm)
    idx = np.where(cos_theta > 1)[0]
    cos_theta[idx] = 1
    theta = np.arccos(cos_theta)
    MAE = np.mean(theta)
    return MAE

In [20]:
a = np.array([[1, 0, 0], [1, 0, 0]])
b = np.array([[-1, 0, 0], [1, 0, 0]])
print(scene_flow_MAE_np(a, b))

1.5707963267948966


In [12]:
file_list_test = []
file_list = get_file_list('\\Workspace\\SceneFlow\\datasets\\argoverse_flow2')
for file in file_list:
    file_list_test.append(file)

In [13]:
EPE_all = []
acc1_all = []
acc2_all = []
acc3_all = []
MAE_all = []

for i, file in enumerate(file_list_test):
    print(i)
    pc_data = np.load(file)
    flow_gt = np.asarray(pc_data['flow_gt'])
    flow_estimate = np.asarray(pc_data['flow_estimate'])
    
    EPE, acc1, acc2, acc3 = scene_flow_EPE_np(flow_estimate, flow_gt)
    MAE = scene_flow_MAE_np(flow_estimate, flow_gt)
    
    EPE_all.append(EPE)
    acc1_all.append(acc1)
    acc2_all.append(acc2)
    acc3_all.append(acc3)
    MAE_all.append(MAE)
    print(EPE, acc1, acc2, acc3, MAE)

0
0.30497182745829565 0.099853515625 0.40594482421875 0.628662109375 0.2610339796621528
1
0.6866545689007817 0.12127685546875 0.2730712890625 0.74468994140625 0.282044321798371
2
0.15509306196768427 0.1956787109375 0.55780029296875 0.52142333984375 0.12389049121482486
3
0.13186197022735419 0.163818359375 0.51611328125 0.56341552734375 0.13275033853938595
4
0.08878709388110871 0.236328125 0.807373046875 0.21728515625 0.07369304832040253
5
0.06251014499932295 0.437744140625 0.97576904296875 0.02716064453125 0.04207595498749503
6
0.03313670944552215 0.97088623046875 0.99249267578125 0.0123291015625 0.027783291162328833
7
0.17268064348382264 0.0008544921875 0.25897216796875 0.74102783203125 0.1425350841653079
8
0.22779729355943917 0.139892578125 0.38006591796875 0.61993408203125 0.20155338915236215
9
0.10238132004288122 0.34130859375 0.938720703125 0.061279296875 0.03336128846425224
10
0.09518504469794209 0.10052490234375 0.79779052734375 0.20220947265625 0.0599361673078278
11
0.1186839752

0.10471772804740008 0.33343505859375 0.64263916015625 0.656494140625 0.17605441949948408
102
0.09086934916262587 0.2999267578125 0.61761474609375 0.7337646484375 0.16502030750317997
103
0.17057789905420412 0.0679931640625 0.33477783203125 0.90631103515625 0.2726215446816356
104
0.07888052755666365 0.45635986328125 0.888671875 0.5394287109375 0.10848549683444106
105
0.07126241096811425 0.43304443359375 0.93304443359375 0.61419677734375 0.19815762979987087
106
0.05705189308515843 0.54974365234375 0.92779541015625 0.91351318359375 0.10837897668819488
107
0.16167653300536264 0.308349609375 0.5946044921875 0.6278076171875 0.195181765584097
108
0.05919446276824942 0.548095703125 0.95660400390625 0.1556396484375 0.10328503809863002
109
0.09466393521730146 0.08837890625 0.61993408203125 0.98602294921875 0.20322027050698666
110
0.08860019463011862 0.37274169921875 0.7816162109375 1.0 0.5393778891274967
111
0.14825487599631051 0.41900634765625 0.5908203125 0.998779296875 1.5420304645402625
112
0

In [14]:
print(np.mean(EPE_all), np.mean(acc1_all), np.mean(acc2_all), np.mean(acc3_all), np.mean(MAE_all))

0.13397259449172802 0.39832780945975826 0.7204477562094634 0.554658997733638 0.39810089007340393
